In [5]:
DIRS = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]

def in_bounds(x,y,n):
    return 0 <= x < n and 0 <= y < n

def heuristic(a,b,method='euclidean'):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    if method == 'manhattan':
        return abs(dx) + abs(dy)
    return (dx*dx + dy*dy) ** 0.5

def best_first_search_grid(grid, heuristic_type='euclidean'):
    n = len(grid)
    start, goal = (0,0), (n-1,n-1)
    if grid[0][0] == 1 or grid[goal[0]][goal[1]] == 1:
        return -1, []
    OPEN = [(start, None, heuristic(start, goal, heuristic_type))]
    CLOSED = []
    seen = set([start])
    parent = {start: None}
    while OPEN:
        OPEN.sort(key=lambda t: t[2])
        N, parentN, hVal = OPEN.pop(0)
        if N == goal:
            path = []
            cur = N
            while cur is not None:
                path.append(cur)
                cur = parent.get(cur)
            return len(path), list(reversed(path))
        CLOSED.append((N, parentN, hVal))
        for dx, dy in DIRS:
            nx, ny = N[0] + dx, N[1] + dy
            if not in_bounds(nx, ny, n):
                continue
            if grid[nx][ny] == 1:
                continue
            M = (nx, ny)
            if M in seen:
                continue
            seen.add(M)
            parent[M] = N
            OPEN.append((M, N, heuristic(M, goal, heuristic_type)))
    return -1, []

def propagate_improvement(node, grid, g, parent, f, goal, heuristic_type):
    n = len(grid)
    queue = [node]
    qi = 0
    while qi < len(queue):
        cur = queue[qi]
        qi += 1
        cur_g = g.get(cur, float('inf'))
        for dx, dy in DIRS:
            nx, ny = cur[0] + dx, cur[1] + dy
            if not in_bounds(nx, ny, n) or grid[nx][ny] == 1:
                continue
            neigh = (nx, ny)
            if cur_g + 1 < g.get(neigh, float('inf')):
                g[neigh] = cur_g + 1
                parent[neigh] = cur
                f[neigh] = g[neigh] + heuristic(neigh, goal, heuristic_type)
                queue.append(neigh)

def a_star_with_propagation(grid, heuristic_type='euclidean'):
    n = len(grid)
    start, goal = (0,0), (n-1,n-1)

    if grid[0][0] == 1 or grid[goal[0]][goal[1]] == 1:
        return -1, []
    g = {}
    f = {}
    parent = {}
    for i in range(n):
        for j in range(n):
            g[(i,j)] = float('inf')
            f[(i,j)] = float('inf')
            parent[(i,j)] = None
    g[start] = 0
    f[start] = heuristic(start, goal, heuristic_type)
    OPEN = [(f[start], 0, start)]
    OPEN_set = set([start])
    CLOSED = set()
    counter = 1
    while OPEN:
        OPEN.sort(key=lambda t: (t[0], t[1]))
        _, _, N = OPEN.pop(0)
        if N in CLOSED:
            continue
        OPEN_set.discard(N)
        CLOSED.add(N)
        if N == goal:
            path = []
            cur = N
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            return len(path), list(reversed(path))
        for dx, dy in DIRS:
            nx, ny = N[0] + dx, N[1] + dy
            if not in_bounds(nx, ny, n) or grid[nx][ny] == 1:
                continue
            M = (nx, ny)
            tentative_g = g[N] + 1
            if tentative_g < g.get(M, float('inf')):
                parent[M] = N
                g[M] = tentative_g
                f[M] = g[M] + heuristic(M, goal, heuristic_type)
                if M in OPEN_set:
                    continue
                if M in CLOSED:
                    propagate_improvement(M, grid, g, parent, f, goal, heuristic_type)
                    continue
                OPEN.append((f[M], counter, M))
                OPEN_set.add(M)
                counter += 1
    return -1, []
def run_examples():
    examples = [
        [[0,1],[1,0]],
        [[0,0,0],[1,1,0],[1,1,0]],
        [[1,0,0],[1,1,0],[1,1,0]]
    ]
    for i, grid in enumerate(examples, 1):
        bfs_len, bfs_path = best_first_search_grid(grid)
        astar_len, astar_path = a_star_with_propagation(grid)
        print("Example {}:".format(i))
        print("Best First Search  → Path length: {}  {}".format(bfs_len, bfs_path if bfs_len!=-1 else ""))
        print("A* Search          → Path length: {} {}".format(astar_len, astar_path if astar_len!=-1 else ""))
        print()
Docstring: Comparison of Best First Search and A* Search — Across the three given testcases, both algorithms produced identical outcomes. In Example 1 and Example 2, each found the shortest paths of lengths 2 and 4. In Example 3, both correctly reported no path since the start was blocked. The key difference is that A* always guarantees the shortest path, while Best-First only succeeds if the heuristic guides it optimally.    print("Across the three given testcases, Best-First Search and A* produced identical outcomes. In Example 1 and Example 2, both algorithms successfully found the shortest paths of lengths 2 and 4. In Example 3, where the start position was blocked, both correctly reported that no path exists. Although their results matched in these cases, the important difference is that A* always guarantees the shortest path when one exists, while Best-First only succeeds when the heuristic happens to guide it along the optimal route.")

if __name__ == "__main__":
    run_examples()


Example 1:
Best First Search  → Path length: 2  [(0, 0), (1, 1)]
A* Search          → Path length: 2 [(0, 0), (1, 1)]

Example 2:
Best First Search  → Path length: 4  [(0, 0), (0, 1), (1, 2), (2, 2)]
A* Search          → Path length: 4 [(0, 0), (0, 1), (1, 2), (2, 2)]

Example 3:
Best First Search  → Path length: -1  
A* Search          → Path length: -1 

